1. Read the input testfile (Pipe delimited) provided as a "Spark RDD" 

2. Remove the Header Record from the RDD

3. Calculate Final_Price:

       Final_Price = (Size * Price_SQ_FT)

4. Save the final rdd asTextfile with three fields

        Property_ID|Location|Final_Price



In [0]:
%fs head "/FileStore/tables/real_estate.txt"

Property_ID|Location|Price|Bedrooms|Bathrooms|Size|Price_SQ_FT|Status
1461262|Arroyo Grande|795000|3|3|2371|365.3|Short Sale
1478004|Paulo Pablo|399000|4|3|2818|163.59|Short Sale
1486551|Paulo Pablo|545000|4|3|3032|179.75|Short Sale
1492832|Santa Bay|909000|4|4|3540|286.78|Short Sale
1499102|Thomas Country|109900|3|1|1249|98.99|Short Sale
1489132|Thomas Country|109000|2|1|1129|93.99|Short Sale
1467262|Fort Worth|987000|4|3|2771|465.3|Short Sale
1478114|Paulo Pablo|409000|4|3|2918|223.19|Short Sale
1402551|Nashville|545000|4|3|2932|169.75|Short Sale
1405832|San Jose|980000|4|4|3340|290.98|Short Sale
1493302|Fort Worth|119900|3|2|2249|198.99|Short Sale
1412332|Thomas Country|129000|3|2|1329|73.99|Short Sale
1469062|Arroyo Grande|798000|3|4|2321|235.9|Short Sale
1498004|Nashville|789000|4|3|2419|263.59|Short Sale
1586751|Nashville|896000|4|3|3132|199.75|Short Sale
1433232|Glendale|987000|4|4|3340|216.78|Short Sale
1495502|Fort Worth|219900|3|2|987|200.99|Short Sale
1489100|San Jose|107200|1|1|789|78.99|Short Sale

In [0]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("demo").getOrCreate()
sc=spark.sparkContext
rdd=sc.textFile("/FileStore/tables/real_estate.txt")

In [0]:
# how to skip first five lines of a text file
rdd1=rdd.zipWithIndex().filter(lambda x: x[1]>=5).map(lambda x: x[0])
rdd1.collect()

Out[5]: ['1499102|Thomas Country|109900|3|1|1249|98.99|Short Sale',
 '1489132|Thomas Country|109000|2|1|1129|93.99|Short Sale',
 '1467262|Fort Worth|987000|4|3|2771|465.3|Short Sale',
 '1478114|Paulo Pablo|409000|4|3|2918|223.19|Short Sale',
 '1402551|Nashville|545000|4|3|2932|169.75|Short Sale',
 '1405832|San Jose|980000|4|4|3340|290.98|Short Sale',
 '1493302|Fort Worth|119900|3|2|2249|198.99|Short Sale',
 '1412332|Thomas Country|129000|3|2|1329|73.99|Short Sale',
 '1469062|Arroyo Grande|798000|3|4|2321|235.9|Short Sale',
 '1498004|Nashville|789000|4|3|2419|263.59|Short Sale',
 '1586751|Nashville|896000|4|3|3132|199.75|Short Sale',
 '1433232|Glendale|987000|4|4|3340|216.78|Short Sale',
 '1495502|Fort Worth|219900|3|2|987|200.99|Short Sale',
 '1489100|San Jose|107200|1|1|789|78.99|Short Sale']

In [0]:
text_header=rdd.filter(lambda x: x.startswith("Property_ID"))
text_file=rdd.filter(lambda x: not x.startswith("Property_ID"))
rdd_textfile=text_file.map(lambda x: x.split('|'))
rdd_textfile.collect()

Out[27]: [['1461262',
  'Arroyo Grande',
  '795000',
  '3',
  '3',
  '2371',
  '365.3',
  'Short Sale'],
 ['1478004',
  'Paulo Pablo',
  '399000',
  '4',
  '3',
  '2818',
  '163.59',
  'Short Sale'],
 ['1486551',
  'Paulo Pablo',
  '545000',
  '4',
  '3',
  '3032',
  '179.75',
  'Short Sale'],
 ['1492832', 'Santa Bay', '909000', '4', '4', '3540', '286.78', 'Short Sale'],
 ['1499102',
  'Thomas Country',
  '109900',
  '3',
  '1',
  '1249',
  '98.99',
  'Short Sale'],
 ['1489132',
  'Thomas Country',
  '109000',
  '2',
  '1',
  '1129',
  '93.99',
  'Short Sale'],
 ['1467262', 'Fort Worth', '987000', '4', '3', '2771', '465.3', 'Short Sale'],
 ['1478114',
  'Paulo Pablo',
  '409000',
  '4',
  '3',
  '2918',
  '223.19',
  'Short Sale'],
 ['1402551', 'Nashville', '545000', '4', '3', '2932', '169.75', 'Short Sale'],
 ['1405832', 'San Jose', '980000', '4', '4', '3340', '290.98', 'Short Sale'],
 ['1493302', 'Fort Worth', '119900', '3', '2', '2249', '198.99', 'Short Sale'],
 ['1412332',
  'Thoma

In [0]:
col_list=text_header.first().split('|')
h1=col_list.index("Property_ID")
h2=col_list.index("Location")
h3=col_list.index("Size")
h4=col_list.index("Price_SQ_FT")

In [0]:
#Function definition to calculate the final price
def mul_price(d1,d2):
    res=float(d1)*float(d2)
    return str(res)

In [0]:
header_output=text_header.map(lambda x : x.split("|")[h1] +"|"+x.split("|")[h2]+"|final_price")
text_output=rdd_textfile.map(lambda x: x[h1]+"|"+x[h2]+"|" +mul_price(x[h3],x[h4]))
text_output.collect()

Out[41]: ['1461262|Arroyo Grande|866126.3',
 '1478004|Paulo Pablo|460996.62',
 '1486551|Paulo Pablo|545002.0',
 '1492832|Santa Bay|1015201.2',
 '1499102|Thomas Country|123638.51',
 '1489132|Thomas Country|106114.70999999999',
 '1467262|Fort Worth|1289346.3',
 '1478114|Paulo Pablo|651268.42',
 '1402551|Nashville|497707.0',
 '1405832|San Jose|971873.2000000001',
 '1493302|Fort Worth|447528.51',
 '1412332|Thomas Country|98332.70999999999',
 '1469062|Arroyo Grande|547523.9',
 '1498004|Nashville|637624.21',
 '1586751|Nashville|625617.0',
 '1433232|Glendale|724045.2',
 '1495502|Fort Worth|198377.13',
 '1489100|San Jose|62323.10999999999']

In [0]:
final_output=header_output.union(text_output)
final_output.collect()

Out[43]: ['Property_ID|Location|final_price',
 '1461262|Arroyo Grande|866126.3',
 '1478004|Paulo Pablo|460996.62',
 '1486551|Paulo Pablo|545002.0',
 '1492832|Santa Bay|1015201.2',
 '1499102|Thomas Country|123638.51',
 '1489132|Thomas Country|106114.70999999999',
 '1467262|Fort Worth|1289346.3',
 '1478114|Paulo Pablo|651268.42',
 '1402551|Nashville|497707.0',
 '1405832|San Jose|971873.2000000001',
 '1493302|Fort Worth|447528.51',
 '1412332|Thomas Country|98332.70999999999',
 '1469062|Arroyo Grande|547523.9',
 '1498004|Nashville|637624.21',
 '1586751|Nashville|625617.0',
 '1433232|Glendale|724045.2',
 '1495502|Fort Worth|198377.13',
 '1489100|San Jose|62323.10999999999']